### PROGRESS: Have been able to successfully access cricinfo and get the runs, ball, players and commentary. I also managed to get the pages to load automatically, and have cleaned and arranged the data to some extent.
#### HOWEVER: I have to try it on other games to see if it is repeatable.
##### Other additions: Full names of Batters and Bowlers + Text analysis using ML on the commentary data
##### Here is the link to the ChatGPT chat that helped me - https://chat.openai.com/c/10f5df72-a242-4251-af0b-c1ce1ed927df

In [70]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [73]:
commentary_selector = 'div.ds-text-tight-m p.ci-html-content'
player_selector = '.ds-leading-none.ds-mb-0\\.5'
ball_selector = 'span.ds-text-tight-s.ds-font-regular.ds-mb-1.lg\:ds-mb-0.lg\:ds-mr-3.ds-block.ds-text-center.ds-text-typo-mid1'
runs_selector = 'div[class^="ds-text-tight-m ds-font-bold ds-flex ds-items-center ds-justify-center ds-text-center ds-w-10 ds-h-10 ds-text-"] span'

## THE ACTUAL CODE

In [72]:
# Set up the Selenium WebDriver
driver = webdriver.Chrome()

# Load the webpage
url = "https://www.espncricinfo.com/series/australia-women-in-india-2023-24-1406065/india-women-vs-australia-women-2nd-odi-1406079/ball-by-ball-commentary"
driver.get(url)

# Wait for some initial content to be present
wait = WebDriverWait(driver, 10)

# Get the initial count of commentary items
initial_count = len(driver.find_elements(By.CSS_SELECTOR, commentary_selector))

# Define the commentary element to use as a reference point for scrolling
commentary_element = driver.find_element(By.CSS_SELECTOR, commentary_selector)

# Scroll to the last commentary item
driver.execute_script("arguments[0].scrollIntoView();", commentary_element)

# Wait until the last commentary item is visible
wait.until(EC.visibility_of(commentary_element))

# Scroll up a bit to move past the advertisements
driver.execute_script("window.scrollBy(0, -300);")

# Scroll down continuously using JavaScript
scroll_script = "window.scrollTo(0, document.body.scrollHeight);"
current_count = initial_count
timeout_duration = 3  # seconds

# Set a counter and a threshold
scroll_counter = 0
max_scroll_attempts = 5  # You can adjust this threshold as needed

while scroll_counter < max_scroll_attempts:
    driver.execute_script(scroll_script)
    
    # Wait for a short duration to allow the scroll to take effect
    time.sleep(2)

    # Get the updated count of commentary items
    new_count = len(driver.find_elements(By.CSS_SELECTOR, commentary_selector))

    # Check if the page has scrolled
    if new_count > current_count:
        current_count = new_count
        scroll_counter = 0  # Reset the counter if new content is loaded
    else:
        # If the page doesn't scroll, increment the counter
        scroll_counter += 1

        # If the counter exceeds the threshold, break out of the loop
        if scroll_counter >= max_scroll_attempts:
            break

        # If the page doesn't scroll, scroll up
        driver.execute_script("window.scrollBy(0, -600);")

        # Wait for a short duration before trying to scroll down again
        time.sleep(2)

# Get the page source after loading all content
html = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extract the commentary text or perform other operations as needed
commentary_items = soup.select(commentary_selector)
balls_items = soup.select(ball_selector)
runs_items = soup.select(runs_selector)
player_items = soup.select(player_selector)

# Create a list of dictionaries to store the data
data = []

for ball, runs, commentary, player in zip(balls_items, runs_items, commentary_items, player_items):
    ball_text = ball.get_text(strip=True)
    runs_text = runs.get_text(strip=True)
    commentary_text = commentary.get_text(strip=True)
    player_text = player.get_text(strip=True)

    data.append({
        'Ball': ball_text,
        'Players': player_text,
        'Runs': runs_text,
        'Commentary': commentary_text
    })

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

# DATA CLEAN UP

# Assuming df is your DataFrame
df['Players'] = df['Players'].str.split(',').str[0]
df[['Bowler', 'Batter']] = df['Players'].str.split(' to ', expand=True)
df.drop(columns=['Players'], inplace=True)

# Replace '•' with '0' in the "Runs" column
df['Runs'] = df['Runs'].replace('•', '0')

# Split the "Ball" column into "Over" and "Ball"
df[['Over', 'Ball']] = df['Ball'].str.split('.', expand=True)

# Fill NaN values with 0 for the "Ball" column
df['Ball'].fillna(0, inplace=True)

# Convert the columns to int64
df[['Over', 'Ball']] = df[['Over', 'Ball']].astype('int64')

# Suppose df is your DataFrame
new_order = ['Over', 'Ball', 'Batter', 'Bowler', 'Runs', 'Commentary']

# Rearrange columns
df = df[new_order]

# Sort the DataFrame based on the 'Over' column
df.sort_values(by=['Over', 'Ball'], inplace=True)

# Reset the index after sorting
df.reset_index(drop=True, inplace=True)

# Print the DataFrame
df

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00007FF646C92142+3514994]
	(No symbol) [0x00007FF6468B0CE2]
	(No symbol) [0x00007FF6467576AA]
	(No symbol) [0x00007FF646730AFD]
	(No symbol) [0x00007FF6467CCB1B]
	(No symbol) [0x00007FF6467E218F]
	(No symbol) [0x00007FF6467C5D93]
	(No symbol) [0x00007FF646794BDC]
	(No symbol) [0x00007FF646795C64]
	GetHandleVerifier [0x00007FF646CBE16B+3695259]
	GetHandleVerifier [0x00007FF646D16737+4057191]
	GetHandleVerifier [0x00007FF646D0E4E3+4023827]
	GetHandleVerifier [0x00007FF6469E04F9+689705]
	(No symbol) [0x00007FF6468BC048]
	(No symbol) [0x00007FF6468B8044]
	(No symbol) [0x00007FF6468B81C9]
	(No symbol) [0x00007FF6468A88C4]
	BaseThreadInitThunk [0x00007FFEC424257D+29]
	RtlUserThreadStart [0x00007FFEC4D2AA58+40]


In [ ]:
df.to_excel("trial.xlsx", index=False)

### PAST RENDITIONS

In [37]:
def scroll_and_extract(driver, selectors, type_names):
    # Wait for some initial content to be present
    wait = WebDriverWait(driver, 10)

    # Create a dictionary to store data for each type
    data = {type_name: [] for type_name in type_names}

    # Iterate over selectors and type names
    for selector, type_name in zip(selectors, type_names):
        # Get the initial count of items
        initial_count = len(driver.find_elements(By.CSS_SELECTOR, selector))

        # Scroll to the bottom of the page to trigger more content loading
        while True:
            driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
            time.sleep(2)  # Add a delay to allow content to load (you may adjust this based on your needs)

            # Get the current count of items
            current_count = len(driver.find_elements(By.CSS_SELECTOR, selector))

            # Break the loop if the count has not increased in the last scroll
            if current_count == initial_count:
                break

            # Update the initial count
            initial_count = current_count

        # Get the page source after loading all content
        html = driver.page_source

        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Extract the text or perform other operations as needed
        items = soup.select(selector)

        for item in items:
            text = item.get_text(strip=True)
            data[type_name].append(text)

    # Convert data to a DataFrame
    df = pd.DataFrame(data)

    return df

In [46]:
# Set up the Selenium WebDriver (make sure to download the appropriate driver for your browser)
driver = webdriver.Chrome()

# Load the webpage
url = "https://www.espncricinfo.com/series/australia-women-in-india-2023-24-1406065/india-women-vs-australia-women-2nd-odi-1406079/ball-by-ball-commentary"
driver.get(url)

# Define selectors and type names
commentary_selector = 'div.ds-text-tight-m p.ci-html-content'
player_selector = '.ds-leading-none.ds-mb-0\\.5'
ball_selector = 'span.ds-text-tight-s.ds-font-regular.ds-mb-1.lg\:ds-mb-0.lg\:ds-mr-3.ds-block.ds-text-center.ds-text-typo-mid1'
runs_selector = 'div[class^="ds-text-tight-m ds-font-bold ds-flex ds-items-center ds-justify-center ds-text-center ds-w-10 ds-h-10 ds-text-"] span'

selectors = [commentary_selector, ball_selector, player_selector, runs_selector]
type_names = ['Commentary', 'Ball', 'Players', 'Runs']

# Call the function with selectors and type names
df = scroll_and_extract(driver, selectors, type_names)

# Display the DataFrame
print(df)

# Close the browser
driver.quit()

                                           Commentary  Ball  \
0   Reverse pulls, beats short third to the right....  49.6   
1   Mathematically impossible unless there are ext...  49.5   
2   Yorker and all Deepti could do is drill it bac...  49.4   
3   Tight wide call, given! Back-of-hand cross seamer  49.4   
4   Dropped by McGrath at cover, that's probably t...  49.3   
5   Deepti plinks the pull and they get only one!....  49.2   
6   Deepti makes room and thrashes it through extr...  49.1   
7   Harleen dances down and Gardner goes through h...  48.6   
8   Makes room and cuts the length ball from aroun...  48.5   
9   Makes room and slaps it hard but finds cover. ...  48.4   
10  Looped up invitingly, beats the big swing. 75k...  48.3   
11  Shortish ball at 81ks keeps low. Harleen goes ...  48.2   
12  Shortish ball outside off is belted straight t...  48.1   
13   Overpitched around off, driven straight to cover  47.6   
14  Wide on a length, 111ks, Harleen tries to cut ...  

In [ ]:
# Assuming df is your DataFrame
df['Players'] = df['Players'].str.split(',').str[0]

# Assuming df is your DataFrame
df[['Bowler', 'Batter']] = df['Players'].str.split(' to ', expand=True)

# Assuming df is your DataFrame
df.drop(columns=['Players'], inplace=True)

print(df)

In [52]:
# Replace '•' with '0' in the "Runs" column
df['Runs'] = df['Runs'].replace('•', '0')

# Display the updated DataFrame
print(df)

                                           Commentary  Ball Runs      Bowler  \
0   Reverse pulls, beats short third to the right....  49.6    4  Sutherland   
1   Mathematically impossible unless there are ext...  49.5    1  Sutherland   
2   Yorker and all Deepti could do is drill it bac...  49.4    0  Sutherland   
3   Tight wide call, given! Back-of-hand cross seamer  49.4   1w  Sutherland   
4   Dropped by McGrath at cover, that's probably t...  49.3    1  Sutherland   
5   Deepti plinks the pull and they get only one!....  49.2    1  Sutherland   
6   Deepti makes room and thrashes it through extr...  49.1    4  Sutherland   
7   Harleen dances down and Gardner goes through h...  48.6    W     Gardner   
8   Makes room and cuts the length ball from aroun...  48.5    1     Gardner   
9   Makes room and slaps it hard but finds cover. ...  48.4    0     Gardner   
10  Looped up invitingly, beats the big swing. 75k...  48.3    0     Gardner   
11  Shortish ball at 81ks keeps low. Har

In [58]:
# Split the "Ball" column into "Over" and "Ball"
df[['Over', 'Ball']] = df['Ball'].str.split('.', expand=True)

# Fill NaN values with 0 for the "Ball" column
df['Ball'].fillna(0, inplace=True)

# Convert the columns to int64
df[['Over', 'Ball']] = df[['Over', 'Ball']].astype('int64')

# Display the updated DataFrame
print(df)

                                           Commentary  Ball Runs      Bowler  \
0   Reverse pulls, beats short third to the right....     6    4  Sutherland   
1   Mathematically impossible unless there are ext...     5    1  Sutherland   
2   Yorker and all Deepti could do is drill it bac...     4    0  Sutherland   
3   Tight wide call, given! Back-of-hand cross seamer     4   1w  Sutherland   
4   Dropped by McGrath at cover, that's probably t...     3    1  Sutherland   
5   Deepti plinks the pull and they get only one!....     2    1  Sutherland   
6   Deepti makes room and thrashes it through extr...     1    4  Sutherland   
7   Harleen dances down and Gardner goes through h...     6    W     Gardner   
8   Makes room and cuts the length ball from aroun...     5    1     Gardner   
9   Makes room and slaps it hard but finds cover. ...     4    0     Gardner   
10  Looped up invitingly, beats the big swing. 75k...     3    0     Gardner   
11  Shortish ball at 81ks keeps low. Har

In [63]:
# Suppose df is your DataFrame
new_order = ['Over', 'Ball', 'Batter', 'Bowler', 'Runs', 'Commentary']

# Rearrange columns
df = df[new_order]

# Sort the DataFrame based on the 'Over' column
df.sort_values(by=['Over', 'Ball'], inplace=True)

# Reset the index after sorting
df.reset_index(drop=True, inplace=True)

df

,Over,Ball,Batter,Bowler,Runs,Commentary
0,42,1,Ghosh,Gardner,1,Length ball lands around fourth and straighten...
1,42,2,Sharma,Gardner,0,"Nice bounce. Offbreak lands around fifth, turn..."
2,42,3,Sharma,Gardner,0,Spin again. Lands on a fullish length in line ...
3,42,4,Sharma,Gardner,1,Makes room and tries to go inside out. Gets th...
4,42,5,Ghosh,Gardner,4,"Ghosh steps leg side, and Gardner goes for the..."
5,42,6,Ghosh,Gardner,1,Turns the length ball drifting down leg to squ...
6,43,1,Ghosh,Sutherland,4,Waits foe the slower one into the pitch and pu...
7,43,2,Ghosh,Sutherland,0,"Cross-seam yorker around off, blocked in front"
8,43,3,Ghosh,Sutherland,1w,"Like the first ball, but it's further down leg..."
9,43,3,Ghosh,Sutherland,1,"Yorker on leg, jammed to the bowler's right, S..."


In [64]:
# Data Clean Up

# Assuming df is your DataFrame
df['Players'] = df['Players'].str.split(',').str[0]
df[['Bowler', 'Batter']] = df['Players'].str.split(' to ', expand=True)
df.drop(columns=['Players'], inplace=True)

# Replace '•' with '0' in the "Runs" column
df['Runs'] = df['Runs'].replace('•', '0')

# Split the "Ball" column into "Over" and "Ball"
df[['Over', 'Ball']] = df['Ball'].str.split('.', expand=True)

# Fill NaN values with 0 for the "Ball" column
df['Ball'].fillna(0, inplace=True)

# Convert the columns to int64
df[['Over', 'Ball']] = df[['Over', 'Ball']].astype('int64')

# Suppose df is your DataFrame
new_order = ['Over', 'Ball', 'Batter', 'Bowler', 'Runs', 'Commentary']

# Rearrange columns
df = df[new_order]

# Sort the DataFrame based on the 'Over' column
df.sort_values(by=['Over', 'Ball'], inplace=True)

# Reset the index after sorting
df.reset_index(drop=True, inplace=True)

print(df)

KeyError: 'Players'

### BELOW IS FIRST DRAFT OF THE CODE, WHERE EVERYTHING WAS SEPARATE

##### Included for historical purposes

In [ ]:
# Set up the Selenium WebDriver (make sure to download the appropriate driver for your browser)
driver = webdriver.Chrome()

# Load the webpage
url = "https://www.espncricinfo.com/series/australia-women-in-india-2023-24-1406065/india-women-vs-australia-women-2nd-odi-1406079/ball-by-ball-commentary"  # Replace with the URL of the cricket commentary page
driver.get(url)

# Wait for some initial content to be present
wait = WebDriverWait(driver, 10)

# Get the initial count of commentary items
initial_count = len(driver.find_elements(By.CSS_SELECTOR, commentary_selector))

# Scroll to the bottom of the page to trigger more content loading
while True:
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(2)  # Add a delay to allow content to load (you may adjust this based on your needs)

    # Get the current count of commentary items
    current_count = len(driver.find_elements(By.CSS_SELECTOR, commentary_selector))

    # Break the loop if the count has not increased in the last scroll
    if current_count == initial_count:
        break

    # Update the initial count
    initial_count = current_count

# Get the page source after loading all content
html = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extract the commentary text or perform other operations as needed
commentary_items = soup.select(commentary_selector)
for item in commentary_items:
    text = item.get_text(strip=True)
    print(text)

In [3]:
def scroll_and_extract(driver, selector):
    # Wait for some initial content to be present
    wait = WebDriverWait(driver, 10)

    # Get the initial count of items
    initial_count = len(driver.find_elements(By.CSS_SELECTOR, selector))

    # Scroll to the bottom of the page to trigger more content loading
    while True:
        driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
        time.sleep(2)  # Add a delay to allow content to load (you may adjust this based on your needs)

        # Get the current count of items
        current_count = len(driver.find_elements(By.CSS_SELECTOR, selector))

        # Break the loop if the count has not increased in the last scroll
        if current_count == initial_count:
            break

        # Update the initial count
        initial_count = current_count

    # Get the page source after loading all content
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the text or perform other operations as needed
    items = soup.select(selector)
    for item in items:
        text = item.get_text(strip=True)
        print(text)

In [5]:
# Set up the Selenium WebDriver (make sure to download the appropriate driver for your browser)
driver = webdriver.Chrome()

# Load the webpage
url = "https://www.espncricinfo.com/series/australia-women-in-india-2023-24-1406065/india-women-vs-australia-women-2nd-odi-1406079/ball-by-ball-commentary"  # Replace with the URL of the cricket commentary page
driver.get(url)

# Call the function with commentary_selector
scroll_and_extract(driver, commentary_selector)

# Call the function with ball_selector
scroll_and_extract(driver, ball_selector)

scroll_and_extract(driver, runs_selector)

# Close the browser
driver.quit()

Reverse pulls, beats short third to the right. Goes to the deep-third fence butIndia fall three agonising runs short!Was a great shot. Shortish ball just outside off, Shreyanka reversed the grip and smacked it along the ground, past the diving fielder
Mathematically impossible unless there are extras now!Into the pitch outside off, looked like the slower one. Deepti slapped it in the air and top edged just past the ring. Long-on came in and stopped it.
Yorker and all Deepti could do is drill it back to the bowler, 9 off 2 needed!It was just outside off, Deepti stayed leg side and smothered it right into the pitch
Tight wide call, given! Back-of-hand cross seamer
Dropped by McGrath at cover, that's probably the 11th for the day and could've closed out the game! She tried to take it above head, clasped at it and it burst through the hands.
Deepti plinks the pull and they get only one!. Slow and very short, doesn't rise much though. Deepti went hard at the front-foot pull and got it to sq

In [28]:
ball_selector = 'span.ds-text-tight-s.ds-font-regular.ds-mb-1.lg\:ds-mb-0.lg\:ds-mr-3.ds-block.ds-text-center.ds-text-typo-mid1'

# Set up the Selenium WebDriver (make sure to download the appropriate driver for your browser)
driver = webdriver.Chrome()

# Load the webpage
url = "https://www.espncricinfo.com/series/australia-women-in-india-2023-24-1406065/india-women-vs-australia-women-2nd-odi-1406079/ball-by-ball-commentary"  # Replace with the URL of the cricket commentary page
driver.get(url)

# Wait for some initial content to be present
wait = WebDriverWait(driver, 10)

# Get the initial count of commentary items
initial_count = len(driver.find_elements(By.CSS_SELECTOR, ball_selector))

# Scroll to the bottom of the page to trigger more content loading
while True:
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(3)  # Add a delay to allow content to load (you may adjust this based on your needs)

    # Get the current count of commentary items
    current_count = len(driver.find_elements(By.CSS_SELECTOR, ball_selector))

    # Break the loop if the count has not increased in the last scroll
    if current_count == initial_count:
        break

    # Update the initial count
    initial_count = current_count

# Get the page source after loading all content
html = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extract the commentary text or perform other operations as needed
commentary_items = soup.select(ball_selector)
for item in commentary_items:
    text = item.get_text(strip=True)
    print(text)

49.6
49.5
49.4
49.4
49.3
49.2
49.1
48.6
48.5
48.4
48.3
48.2
48.1
47.6
47.5
47.4
47.3
47.2
47.1
46.6
46.5
46.4
46.3
46.2
46.1
45.6
45.5
45.5
45.5
45.4
45.3
45.2
45.1
44.6
44.5
44.4
44.3
44.2
44.1
43.6
43.5
43.4
43.3
43.3
43.2
43.1
42.6
42.5
42.4
42.3
42.2
42.1


In [30]:
runs_selector = 'div[class^="ds-text-tight-m ds-font-bold ds-flex ds-items-center ds-justify-center ds-text-center ds-w-10 ds-h-10 ds-text-"] span'

# Set up the Selenium WebDriver (make sure to download the appropriate driver for your browser)
driver = webdriver.Chrome()

# Load the webpage
url = "https://www.espncricinfo.com/series/australia-women-in-india-2023-24-1406065/india-women-vs-australia-women-2nd-odi-1406079/ball-by-ball-commentary"  # Replace with the URL of the cricket commentary page
driver.get(url)

# Wait for some initial content to be present
wait = WebDriverWait(driver, 10)

# Get the initial count of commentary items
initial_count = len(driver.find_elements(By.CSS_SELECTOR, runs_selector))

# Scroll to the bottom of the page to trigger more content loading
while True:
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(3)  # Add a delay to allow content to load (you may adjust this based on your needs)

    # Get the current count of commentary items
    current_count = len(driver.find_elements(By.CSS_SELECTOR, runs_selector))

    # Break the loop if the count has not increased in the last scroll
    if current_count == initial_count:
        break

    # Update the initial count
    initial_count = current_count

# Get the page source after loading all content
html = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extract the commentary text or perform other operations as needed
commentary_items = soup.select(runs_selector)
for item in commentary_items:
    text = item.get_text(strip=True)
    print(text)

4
1
•
1w
1
1
4
W
1
•
•
1
1
•
•
W
1
1
1
•
4
1
•
1
1
•
1
1w
1w
1
1
1
W
•
1
1
1
1
1
1
W
1
1
1w
•
4
1
4
1
•
•
1


In [41]:
# Set up the Selenium WebDriver
driver = webdriver.Chrome()
url = "https://www.espncricinfo.com/series/australia-women-in-india-2023-24-1406065/india-women-vs-australia-women-2nd-odi-1406079/ball-by-ball-commentary"
driver.get(url)

# Wait for some initial content to be present
wait = WebDriverWait(driver, 10)
initial_count = len(driver.find_elements(By.XPATH, '//div[contains(@class, "ds-ml-4") and contains(@class, "lg:ds-ml-3") and contains(@class, "ds-text-typo-mid1")]'))

# Scroll to the bottom of the page to trigger more content loading
while True:
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(3)
    current_count = len(driver.find_elements(By.XPATH, '//div[contains(@class, "ds-ml-4") and contains(@class, "lg:ds-ml-3") and contains(@class, "ds-text-typo-mid1")]'))
    
    # Break the loop if the count has not increased in the last scroll
    if current_count == initial_count:
        break

    initial_count = current_count

# Get the page source after loading all content
html = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extract the commentary text or perform other operations as needed
commentary_items = soup.select('div.ds-ml-4.lg\\:ds-ml-3.ds-text-typo-mid1')

for item in commentary_items:
    text_span = item.find('span')
    if text_span:
        text = text_span.get_text(strip=True)
        print(text)

Sutherland to Patil,FOUR runs
Sutherland to Sharma,1 run
Sutherland to Sharma,no run
Sutherland to Sharma,1 wide
Sutherland to Patil,1 run
Sutherland to Sharma,1 run
Sutherland to Sharma,FOUR runs
Gardner to Deol,OUT
Gardner to Sharma,1 run
Gardner to Sharma,no run
Gardner to Sharma,no run
Gardner to Deol,1 run
Gardner to Sharma,1 run
Sutherland to Deol,no run
Sutherland to Deol,no run
Sutherland to Vastrakar,OUT
Sutherland to Sharma,1 run
Sutherland to Vastrakar,1 run
Sutherland to Sharma,1 run
Gardner to Vastrakar,no run
Gardner to Vastrakar,FOUR runs
Gardner to Sharma,1 run
Gardner to Sharma,no run
Gardner to Vastrakar,1 run
Gardner to Sharma,1 run
Sutherland to Vastrakar,no run
Sutherland to Sharma,1 run
Sutherland to Sharma,1 wide
Sutherland to Sharma,1 wide
Sutherland to Vastrakar,1 run
Sutherland to Sharma,1 run
Sutherland to Vastrakar,1 run
Sutherland to Amanjot,OUT
Garth to Sharma,no run
Garth to Amanjot,1 run
Garth to Sharma,1 run
Garth to Amanjot,1 run
Garth to Sharma,1 run
